In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
import time

df_train_raw = pd.read_csv('data/train.csv')
df_test_raw = pd.read_csv('data/test.csv')

def preprocess_init(df_train, df_test, bool_features):

    df = pd.concat([df_train, df_test], sort=True)

    df['days'] = np.select([df['locdt']<=30, [(df['locdt']>30) & (df['locdt']<=60)] , [(df['locdt']>60) & (df['locdt']<=90)], [(df['locdt']>90) & (df['locdt']<=120)]],[30,60,90,120])[0]

    df = preprocess_bool(df, bool_features)

    df = df.sort_values(by = ['bacno','cano','locdt','loctm']).reset_index(drop = True)

    return df
                
def preprocess_bool(df, bool_features):
    
    for feature in bool_features:
        df[feature] = np.select([df[feature]=='Y',df[feature]=='N'],[1,0]) 
    
    return df
def preprocess_time(df):

    df['global_time'] = loctm_to_global_time(df)
    df['last_time_days'] = df.groupby(['cano','days'])['global_time'].diff(periods = 1)
    df['next_time_days'] = df.groupby(['cano','days'])['global_time'].diff(periods = -1)

    groups = ['cano','locdt']
    feature = 'global_time'
    agg_list = [np.std]
    df = generic_groupby(df, groups, feature, agg_list)
    
    return df

def loctm_to_global_time(df):
    
    df = df.copy()
    df['loctm'] = df['loctm'].astype(str)
    df['loctm'] = df['loctm'].str[:-2]
    df['hours'] = df['loctm'].str[-6:-4]
    df['hours'] = np.where(df['hours']=='', '0', df['hours']).astype(int)
    df['minutes'] = df['loctm'].str[-4:-2]
    df['minutes'] = np.where(df['minutes']=='', '0', df['minutes']).astype(int)
    df['second'] = df['loctm'].str[-2:].astype(int)
    df['loctm'] = df['hours']*60*60 + df['minutes']*60 + df['second']
    df['global_time'] = df['locdt']*24*60*60 + df['hours']*60*60+df['minutes']*60+df['second']
                        
    return df['global_time']

def preprocess_train_test_split(df, cat_features):


    df_train = df[~df['fraud_ind'].isna()]
    df_test = df[df['fraud_ind'].isna()]

    # Keep the original value of some feature before replacing
    keep_list = ['cano',
                 'bacno',
                 'mchno'
                 ]
    for keep in keep_list:
        df_train[keep + '_original'] = df_train[keep].copy()
    
    for feature in cat_features:
        test_unique = df_test[feature].unique()
        df_train[feature] = np.where(df_train[feature].isin(test_unique), df_train[feature], np.nan)

    df_test[cat_features] = df_test[cat_features].astype('category')
    df_train[cat_features] = df_train[cat_features].astype('category')

    return df_train, df_test
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

def generic_groupby(df, group, feature, agg_list):
    
    df_tem = df.groupby(group)[feature].agg(agg_list).reset_index()
    agg_list = ['std' if x==np.std else x for x in agg_list]                
    rename_dict = dict([(x,'{}_{}_{}'.format('_'.join(group), feature, x)) for x in agg_list])
    df_tem = df_tem.rename(columns = rename_dict)
    df = pd.merge(df, df_tem, how = 'left', on = group)

    return df

def applyParallel(dfGrouped, func):
    retLst = Parallel(n_jobs=8)(delayed(func)(group) for name, group in dfGrouped)
    return pd.concat(retLst)

In [2]:
raw_bool_features = ['ecfg',
                     'flbmk',
                     'flg_3dsmk',
                     'insfg',
                     'ovrlt'
                      ]

raw_categorial_features = ['contp',
                           'stscd',
                           'etymd',
                           'stocn',
                           'mcc',
                           'csmcu',
                           'hcefg',
                           'bacno',
                           'cano',
                           'mchno',
                           'acqic',
                           'scity'
                            ] 

raw_contiuous_feautres = ['loctm',
                          'conam',
                          'iterm'
                           ]

In [3]:
df = preprocess_init(df_train=df_train_raw, df_test=df_test_raw, bool_features=raw_bool_features)

In [4]:
df = preprocess_time(df)

In [5]:
#　先製作出絕對時間
df['absolute_time'] =df[['global_time']]+3600*16-29#常數是為了控制時間從０點開始
# min : 1970-01-03 00:00:00 , max : 1970-05-02 23:59:16

In [6]:
#絕對時間轉換成date time 格式
abt = []
bt = df['absolute_time'].values.tolist()
for i in range(len(bt)):
    t = bt[i]
    time_local = time.localtime(t)
    abt.append(time.strftime("%Y-%m-%d %H:%M:%S",time_local))

In [7]:
df['abt'] = abt

In [8]:
h = df.abt.tolist()
hh = []
for i in h:
    hh.append(pd.Timestamp(i))#要換成time stamp的型態才可以rolling
df.index = hh

In [9]:
df['count'] =1

# bacno , cano 排序下一定時間內的特定類別(國別)不同數量
new3_x_bacno_cano_category_count

In [37]:
# 訓練前要先轉好排列順序
df_training_features = df.sort_values(by=['bacno' , 'cano'])

In [ ]:
# stocn n天內該張卡號刷卡交易經歷了多少個國家
#2d
x = df.groupby(['bacno' , 'cano'])['stocn'].rolling('2d',
                                   min_periods=1).apply(lambda x :len(np.unique(x)) )
conam_x =x.tolist()
df_training_features['bacno_cano_stocn_unique_2d'] =conam_x

#6h
x = df.groupby(['bacno','cano'])['stocn'].rolling('6h',
                                    min_periods=1).apply(lambda x :len(np.unique(x)) )
conam_x =x.tolist()
df_training_features['bacno_cano_stocn_unique_6h'] =conam_x

In [44]:
# mchno
#2d
x = df.groupby(['bacno' , 'cano'])['mchno'].rolling('2d',min_periods=1).apply(lambda x :len(np.unique(x)) )
conam_x =x.tolist()
df_training_features['bacno_cano_mchno_unique_2d'] =conam_x
#6h
x = df.groupby(['bacno','cano'])['mchno'].rolling('6h',min_periods=1).apply(lambda x :len(np.unique(x)) )
conam_x =x.tolist()
df_training_features['bacno_cano_mchno_unique_6h'] =conam_x

C:\Users\User\Anaconda3\envs\TK\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\envs\TK\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


In [45]:
#mcc
#2d
x = df.groupby(['bacno' , 'cano'])['mcc'].rolling('2d',min_periods=1).apply(lambda x :len(np.unique(x)) )
conam_x =x.tolist()
df_training_features['bacno_cano_mcc_unique_2d'] =conam_x
#6h
x = df.groupby(['bacno','cano'])['mcc'].rolling('6h',min_periods=1).apply(lambda x :len(np.unique(x)) )
conam_x =x.tolist()
df_training_features['bacno_cano_mcc_unique_6h'] =conam_x

C:\Users\User\Anaconda3\envs\TK\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\envs\TK\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


In [46]:
df_training_features.columns

Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey',
       'days', 'global_time', 'last_time_days', 'next_time_days',
       'cano_locdt_global_time_std', 'absolute_time', 'abt', 'count',
       'bacno_cano_stocn_unique_2d', 'bacno_cano_stocn_unique_6h',
       'bacno_cano_mchno_unique_2d', 'bacno_cano_mchno_unique_6h',
       'bacno_cano_mcc_unique_2d', 'bacno_cano_mcc_unique_6h'],
      dtype='object')

In [47]:
df_training_features[['txkey','bacno_cano_stocn_unique_2d', 'bacno_cano_stocn_unique_6h',
       'bacno_cano_mchno_unique_2d', 'bacno_cano_mchno_unique_6h',
       'bacno_cano_mcc_unique_2d', 'bacno_cano_mcc_unique_6h']].to_csv('new3_x_bacno_cano_category_count.csv')

# bacno_cano排序下的金額與次數
描繪消費者跟過去消費行為間的差距<br>
new_x_bacno_cano_conam

In [73]:
# 訓練前要先轉好排列順序
df_training_features = df.sort_values(by=['bacno' , 'cano'])

In [74]:
#1h
x = df.groupby(['bacno' , 'cano'])['conam'].rolling('1h', min_periods=1).mean()
y = df.groupby(['bacno' , 'cano'])['count'].rolling('1h', min_periods=1).sum()
conam_x =x.tolist()
count_y =y.tolist()
df_training_features['bacno_cano_conam_mean_1h'] =conam_x
df_training_features['bacno_cano_count_sum_1h'] =count_y

In [75]:
#6h
x = df.groupby(['bacno' , 'cano'])['conam'].rolling('6h', min_periods=1).mean()
y = df.groupby(['bacno' , 'cano'])['count'].rolling('6h', min_periods=1).sum()

conam_x =x.tolist()
count_y =y.tolist()
df_training_features['bacno_cano_conam_mean_6h'] =conam_x
df_training_features['bacno_cano_count_sum_6h'] =count_y

In [76]:
#1d
x = df.groupby(['bacno' , 'cano'])['conam'].rolling('1d', min_periods=1).mean()
y = df.groupby(['bacno' , 'cano'])['count'].rolling('1d', min_periods=1).sum()

conam_x =x.tolist()
count_y =y.tolist()
df_training_features['bacno_cano_conam_mean_1d'] =conam_x
df_training_features['bacno_cano_count_sum_1d'] =count_y

In [78]:
#7d
x = df.groupby(['bacno' , 'cano'])['conam'].rolling('7d', min_periods=1).mean()
y = df.groupby(['bacno' , 'cano'])['count'].rolling('7d', min_periods=1).sum()

conam_x =x.tolist()
count_y =y.tolist()
df_training_features['bacno_cano_conam_mean_7d'] =conam_x
df_training_features['bacno_cano_count_sum_7d'] =count_y

In [77]:
#30d
x = df.groupby(['bacno' , 'cano'])['conam'].rolling('30d', min_periods=1).mean()
y = df.groupby(['bacno' , 'cano'])['count'].rolling('30d', min_periods=1).sum()

conam_x =x.tolist()
count_y =y.tolist()
df_training_features['bacno_cano_conam_mean_30d'] =conam_x
df_training_features['bacno_cano_count_sum_30d'] =count_y

In [79]:
#120d
x = df.groupby(['bacno' , 'cano'])['conam'].rolling('120d', min_periods=1).mean()
y = df.groupby(['bacno' , 'cano'])['count'].rolling('120d', min_periods=1).sum()

conam_x =x.tolist()
count_y =y.tolist()
df_training_features['bacno_cano_conam_mean_120d'] =conam_x
df_training_features['bacno_cano_count_sum_120d'] =count_y

In [87]:
# 帳戶卡號的消費在不同時間的消費差距
df_training_features['bacno_cano_conam_mean_1h_xg_conam']=df_training_features['conam']-df_training_features['bacno_cano_conam_mean_1h']
df_training_features['bacno_cano_conam_mean_6h_xg_conam']=df_training_features['conam']-df_training_features['bacno_cano_conam_mean_6h']
df_training_features['bacno_cano_conam_mean_1d_xg_conam']=df_training_features['conam']-df_training_features['bacno_cano_conam_mean_1d']
df_training_features['bacno_cano_conam_mean_7d_xg_conam']=df_training_features['conam']-df_training_features['bacno_cano_conam_mean_7d']
df_training_features['bacno_cano_conam_mean_30d_xg_conam']=df_training_features['conam']-df_training_features['bacno_cano_conam_mean_30d']
df_training_features['bacno_cano_conam_mean_120d_xg_conam']=df_training_features['conam']-df_training_features['bacno_cano_conam_mean_120d']

In [92]:
# 帳戶卡號的消費在不同時間的次數差距
df_training_features['bacno_cano_count_sum_7d_xg_1h']=df_training_features['bacno_cano_count_sum_7d']-df_training_features['bacno_cano_count_sum_1h']
df_training_features['bacno_cano_count_sum_1d_xg_1h']=df_training_features['bacno_cano_count_sum_1d']-df_training_features['bacno_cano_count_sum_1h']
df_training_features['bacno_cano_count_sum_6h_xg_1h']=df_training_features['bacno_cano_count_sum_6h']-df_training_features['bacno_cano_count_sum_1h']


In [101]:
# 帳戶卡號的消費在不同時間的金額比例
df_training_features['bacno_cano_conam_mean_1h_div_conam']=df_training_features['conam']/(df_training_features['bacno_cano_conam_mean_1h']+0.00000001)
df_training_features['bacno_cano_conam_mean_6h_div_conam']=df_training_features['conam']/(df_training_features['bacno_cano_conam_mean_6h']+0.00000001)
df_training_features['bacno_cano_conam_mean_1d_div_conam']=df_training_features['conam']/(df_training_features['bacno_cano_conam_mean_1d']+0.00000001)
df_training_features['bacno_cano_conam_mean_7d_div_conam']=df_training_features['conam']/(df_training_features['bacno_cano_conam_mean_7d']+0.00000001)
df_training_features['bacno_cano_conam_mean_30d_div_conam']=df_training_features['conam']/(df_training_features['bacno_cano_conam_mean_30d']+0.00000001)
df_training_features['bacno_cano_conam_mean_120d_div_conam']=df_training_features['conam']/(df_training_features['bacno_cano_conam_mean_120d']+0.00000001)


In [104]:
# 增加新的特徵
df_training_features[['txkey','bacno_cano_conam_mean_1h', 'bacno_cano_count_sum_1h',
       'bacno_cano_conam_mean_6h', 'bacno_cano_count_sum_6h',
       'bacno_cano_conam_mean_1d', 'bacno_cano_count_sum_1d',
       'bacno_cano_conam_mean_30d', 'bacno_cano_count_sum_30d',
       'bacno_cano_conam_mean_7d', 'bacno_cano_count_sum_7d',
       'bacno_cano_conam_mean_120d', 'bacno_cano_count_sum_120d',
       'bacno_cano_conam_mean_1h_xg_conam',
       'bacno_cano_conam_mean_6h_xg_conam',
       'bacno_cano_conam_mean_1d_xg_conam',
       'bacno_cano_conam_mean_7d_xg_conam',
       'bacno_cano_conam_mean_30d_xg_conam',
       'bacno_cano_conam_mean_120d_xg_conam', 'bacno_cano_count_sum_7d_xg_1h',
       'bacno_cano_count_sum_1d_xg_1h', 'bacno_cano_count_sum_6h_xg_1h',
       'bacno_cano_conam_mean_1h_div_conam',
       'bacno_cano_conam_mean_6h_div_conam',
       'bacno_cano_conam_mean_1d_div_conam',
       'bacno_cano_conam_mean_7d_div_conam',
       'bacno_cano_conam_mean_30d_div_conam',
       'bacno_cano_conam_mean_120d_div_conam']].to_csv('new_x_bacno_cano_conam.csv')

# cano_mchno排序下的金額與次數
new2_x_cano_mhcno_conam

In [13]:
df_training_features = df.sort_values(by=['cano','mchno'])

In [14]:
#1h
x = df.groupby(['cano','mchno'])['conam'].rolling('1h', min_periods=1).mean()
y = df.groupby(['cano','mchno'])['count'].rolling('1h', min_periods=1).sum()
conam_x =x.tolist()
count_y =y.tolist()
df_training_features['cano_mhcno_conam_mean_1h'] =conam_x
df_training_features['cano_mhcno_count_sum_1h'] =count_y

In [15]:
#6h
x = df.groupby(['cano','mchno'])['conam'].rolling('6h', min_periods=1).mean()
y = df.groupby(['cano','mchno'])['count'].rolling('6h', min_periods=1).sum()
conam_x =x.tolist()
count_y =y.tolist()
df_training_features['cano_mhcno_conam_mean_6h'] =conam_x
df_training_features['cano_mhcno_count_sum_6h'] =count_y

In [16]:
#1d
x = df.groupby(['cano','mchno'])['conam'].rolling('1d', min_periods=1).mean() 
y = df.groupby(['cano','mchno'])['count'].rolling('1d', min_periods=1).sum() 
conam_x =x.tolist() 
count_y =y.tolist() 
df_training_features['cano_mhcno_conam_mean_1d'] = conam_x
df_training_features['cano_mhcno_count_sum_1d']  = count_y

In [17]:
#7d
x = df.groupby(['cano','mchno'])['conam'].rolling('7d', min_periods=1).mean() 
y = df.groupby(['cano','mchno'])['count'].rolling('7d', min_periods=1).sum() 
conam_x =x.tolist() 
count_y =y.tolist() 
df_training_features['cano_mhcno_conam_mean_7d'] = conam_x
df_training_features['cano_mhcno_count_sum_7d']  = count_y

In [18]:
#30d
x = df.groupby(['cano','mchno'])['conam'].rolling('30d', min_periods=1).mean() 
y = df.groupby(['cano','mchno'])['count'].rolling('30d', min_periods=1).sum() 
conam_x =x.tolist() 
count_y =y.tolist() 
df_training_features['cano_mhcno_conam_mean_30d'] = conam_x
df_training_features['cano_mhcno_count_sum_30d']  = count_y

In [19]:
#120d
x = df.groupby(['cano','mchno'])['conam'].rolling('120d', min_periods=1).mean() 
y = df.groupby(['cano','mchno'])['count'].rolling('120d', min_periods=1).sum() 
conam_x =x.tolist() 
count_y =y.tolist() 
df_training_features['cano_mhcno_conam_mean_120d'] = conam_x
df_training_features['cano_mhcno_count_sum_120d']  = count_y

In [20]:
## 卡號特電的消費在不同時間的消費差距
df_training_features['cano_mchno_conam_mean_1h_xg_conam']=df_training_features['conam']-df_training_features['cano_mhcno_conam_mean_1h']
df_training_features['cano_mchno_conam_mean_6h_xg_conam']=df_training_features['conam']-df_training_features['cano_mhcno_conam_mean_6h']
df_training_features['cano_mchno_conam_mean_1d_xg_conam']=df_training_features['conam']-df_training_features['cano_mhcno_conam_mean_1d']
df_training_features['cano_mchno_conam_mean_7d_xg_conam']=df_training_features['conam']-df_training_features['cano_mhcno_conam_mean_7d']
df_training_features['cano_mchno_conam_mean_30d_xg_conam']=df_training_features['conam']-df_training_features['cano_mhcno_conam_mean_30d']
df_training_features['cano_mchno_conam_mean_120d_xg_conam']=df_training_features['conam']-df_training_features['cano_mhcno_conam_mean_120d']

In [21]:
# 卡號特電的消費在不同時間的次數差距
df_training_features['cano_mhcno_count_sum_7d_xg_1h']=df_training_features['cano_mhcno_count_sum_7d']-df_training_features['cano_mhcno_count_sum_1h']
df_training_features['cano_mhcno_count_sum_1d_xg_1h']=df_training_features['cano_mhcno_count_sum_1d']-df_training_features['cano_mhcno_count_sum_1h']
df_training_features['cano_mhcno_count_sum_6h_xg_1h']=df_training_features['cano_mhcno_count_sum_6h']-df_training_features['cano_mhcno_count_sum_1h']


In [22]:
# 卡號特電的消費在不同時間的金額比例
df_training_features['cano_mhcno_conam_mean_1h_div_conam']=df_training_features['conam']/(df_training_features['cano_mhcno_conam_mean_1h']+0.00000001)
df_training_features['cano_mhcno_conam_mean_6h_div_conam']=df_training_features['conam']/(df_training_features['cano_mhcno_conam_mean_6h']+0.00000001)
df_training_features['cano_mhcno_conam_mean_1d_div_conam']=df_training_features['conam']/(df_training_features['cano_mhcno_conam_mean_1d']+0.00000001)
df_training_features['cano_mhcno_conam_mean_7d_div_conam']=df_training_features['conam']/(df_training_features['cano_mhcno_conam_mean_7d']+0.00000001)
df_training_features['cano_mhcno_conam_mean_30d_div_conam']=df_training_features['conam']/(df_training_features['cano_mhcno_conam_mean_30d']+0.00000001)
df_training_features['cano_mhcno_conam_mean_120d_div_conam']=df_training_features['conam']/(df_training_features['cano_mhcno_conam_mean_120d']+0.00000001)


In [24]:
df_training_features.columns

Index(['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'fraud_ind', 'hcefg', 'insfg', 'iterm', 'locdt',
       'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'txkey',
       'days', 'global_time', 'last_time_days', 'next_time_days',
       'cano_locdt_global_time_std', 'absolute_time', 'abt', 'count',
       'cano_mhcno_conam_mean_1h', 'cano_mhcno_count_sum_1h',
       'cano_mhcno_conam_mean_6h', 'cano_mhcno_count_sum_6h',
       'cano_mhcno_conam_mean_1d', 'cano_mhcno_count_sum_1d',
       'cano_mhcno_conam_mean_7d', 'cano_mhcno_count_sum_7d',
       'cano_mhcno_conam_mean_30d', 'cano_mhcno_count_sum_30d',
       'cano_mhcno_conam_mean_120d', 'cano_mhcno_count_sum_120d',
       'cano_mchno_conam_mean_1h_xg_conam',
       'cano_mchno_conam_mean_6h_xg_conam',
       'cano_mchno_conam_mean_1d_xg_conam',
       'cano_mchno_conam_mean_7d_xg_conam',
       'cano_mchno_conam_mean_30d_xg_conam',
       'cano_mchno_conam_mean_120

In [25]:
df_training_features[['txkey','cano_mhcno_conam_mean_1h', 'cano_mhcno_count_sum_1h',
       'cano_mhcno_conam_mean_6h', 'cano_mhcno_count_sum_6h',
       'cano_mhcno_conam_mean_1d', 'cano_mhcno_count_sum_1d',
       'cano_mhcno_conam_mean_7d', 'cano_mhcno_count_sum_7d',
       'cano_mhcno_conam_mean_30d', 'cano_mhcno_count_sum_30d',
       'cano_mhcno_conam_mean_120d', 'cano_mhcno_count_sum_120d',
       'cano_mchno_conam_mean_1h_xg_conam',
       'cano_mchno_conam_mean_6h_xg_conam',
       'cano_mchno_conam_mean_1d_xg_conam',
       'cano_mchno_conam_mean_7d_xg_conam',
       'cano_mchno_conam_mean_30d_xg_conam',
       'cano_mchno_conam_mean_120d_xg_conam', 'cano_mhcno_count_sum_7d_xg_1h',
       'cano_mhcno_count_sum_1d_xg_1h', 'cano_mhcno_count_sum_6h_xg_1h',
       'cano_mhcno_conam_mean_1h_div_conam',
       'cano_mhcno_conam_mean_6h_div_conam',
       'cano_mhcno_conam_mean_1d_div_conam',
       'cano_mhcno_conam_mean_7d_div_conam',
       'cano_mhcno_conam_mean_30d_div_conam',
       'cano_mhcno_conam_mean_120d_div_conam']].to_csv('new2_x_cano_mhcno_conam.csv')